In [1]:
# ============================================================
# Notebook único: cálculo + escrita Delta (sem mudar particionamento)
# - Se a tabela não existir: cria sem partição (compatível com seu estado atual)
# - Se já existir: faz MERGE (upsert) pelos campos-chave (DATA, SITUACAO, DESCRICAO_SITUACAO)
# ============================================================
from pyspark.sql import functions as F
from pyspark.sql.utils import AnalysisException
from delta.tables import DeltaTable
import time

# -----------------------------
# CONFIGURAÇÕES
# -----------------------------
path_destino = "abfss://ws_departamento_pessoal@onelake.dfs.fabric.microsoft.com/lk_departamento_pessoal.Lakehouse/Tables/tab_gold_fato_funcionario_situacao_quantidade"

# (Opcional) Nome lógico no catálogo; deixe None para pular
catalog_table_name = None  # Ex.: "lk_departamento_pessoal.tab_gold_fato_funcionario_situacao_quantidade"

MAX_RETRIES = 3
BASE_SLEEP  = 5  # segundos (backoff linear)

# -----------------------------
# CONSULTA (sua lógica do último dia do mês)
# -----------------------------
sql_query = """
WITH UltimoDiaMes AS (
    SELECT 
        CAST(DATE_TRUNC('MONTH', DATA) AS DATE) AS PrimeiroDiaMes,
        MAX(DATA) AS UltimaDataMes
    FROM tab_gold_dim_funcionario_historico
    GROUP BY DATE_TRUNC('MONTH', DATA)
)
SELECT 
    U.UltimaDataMes AS DATA,
    FH.SITUACAO,
    FH.DESCRICAO_SITUACAO,
    COUNT(*) AS QUANTIDADE_FUNCIONARIO
FROM UltimoDiaMes U
JOIN tab_gold_dim_funcionario_historico FH
    ON FH.DATA = U.UltimaDataMes
GROUP BY 
    U.UltimaDataMes,
    FH.SITUACAO,
    FH.DESCRICAO_SITUACAO
"""
df_resultado = spark.sql(sql_query)
df_resultado.cache()
print("Prévia do resultado:")
df_resultado.show(20, truncate=False)

# -----------------------------
# HELPERS
# -----------------------------
def delta_table_exists(path: str) -> bool:
    try:
        DeltaTable.forPath(spark, path)
        return True
    except Exception:
        return False

def is_concurrent_exc(e: Exception) -> bool:
    s = str(e)
    return ("ConcurrentAppendException" in s
            or "checkForConflicts" in s
            or "conflicts" in s)

# -----------------------------
# ESCRITA
#   - Se não existe: cria tabela sem partição (overwrite inicial)
#   - Se existe: MERGE (upsert) por DATA+SITUACAO+DESCRICAO_SITUACAO
# -----------------------------
if not delta_table_exists(path_destino):
    # Criação inicial, mantendo sem particionamento (compatível com o estado reportado)
    print("[INFO] Tabela não existe. Criando (overwrite inicial, sem partição).")
    (df_resultado
        .write
        .format("delta")
        .mode("overwrite")
        .save(path_destino))

    if catalog_table_name:
        spark.sql(f"CREATE TABLE IF NOT EXISTS {catalog_table_name} USING DELTA LOCATION '{path_destino}'")
        print(f"[OK] Tabela registrada no catálogo: {catalog_table_name}")

else:
    print("[INFO] Tabela existe. Executando MERGE (upsert) para evitar overwrite total e manter o particionamento atual.")
    # Cria uma view temporária do source para usar no MERGE SQL ou usa API Python.
    # Aqui vamos pela API Python do DeltaTable.
    dt = DeltaTable.forPath(spark, path_destino)

    # Para garantir idempotência e lidar com concorrência leve, aplicamos retentativa.
    last_err = None
    for attempt in range(1, MAX_RETRIES + 1):
        try:
            (dt.alias("t")
               .merge(
                   df_resultado.alias("s"),
                   "t.DATA = s.DATA "
                   "AND t.SITUACAO = s.SITUACAO "
                   "AND t.DESCRICAO_SITUACAO = s.DESCRICAO_SITUACAO"
               )
               .whenMatchedUpdate(set={
                   "QUANTIDADE_FUNCIONARIO": "s.QUANTIDADE_FUNCIONARIO"
               })
               .whenNotMatchedInsertAll()
               .execute())
            print(f"[OK] MERGE concluído (tentativa {attempt}).")
            last_err = None
            break
        except Exception as e:
            last_err = e
            if is_concurrent_exc(e) and attempt < MAX_RETRIES:
                wait_s = BASE_SLEEP * attempt
                print(f"[Aviso] Concorrência detectada no MERGE (tentativa {attempt}). Aguardando {wait_s}s para retentar...")
                time.sleep(wait_s)
            else:
                print(f"[Erro] MERGE falhou na tentativa {attempt}.")
                raise

    if last_err is None and catalog_table_name:
        spark.sql(f"CREATE TABLE IF NOT EXISTS {catalog_table_name} USING DELTA LOCATION '{path_destino}'")
        print(f"[OK] Tabela confirmada/registrada no catálogo: {catalog_table_name}")

# -----------------------------
# (Opcional) Ver histórico após a escrita
# -----------------------------
try:
    dt_final = DeltaTable.forPath(spark, path_destino)
    print("Histórico Delta (últimos 10 commits):")
    dt_final.history().show(10, truncate=False)
except Exception as e:
    print("Não foi possível ler o histórico após a escrita.")
    print(e)


StatementMeta(, 3a5a0a5d-a0a8-4324-a939-f7cff0cd248e, 3, Finished, Available, Finished)

Prévia do resultado:
+----------+--------+---------------------------------+----------------------+
|DATA      |SITUACAO|DESCRICAO_SITUACAO               |QUANTIDADE_FUNCIONARIO|
+----------+--------+---------------------------------+----------------------+
|2025-06-30|E       |Licença Mater.                   |4                     |
|2025-01-31|V       |Aviso Prévio                     |75                    |
|2020-02-29|F       |Férias                           |108                   |
|2008-08-31|P       |Af.Previdência                   |15                    |
|2008-10-31|P       |Af.Previdência                   |16                    |
|2005-07-31|P       |Af.Previdência                   |7                     |
|2024-04-30|E       |Licença Mater.                   |5                     |
|2023-06-30|E       |Licença Mater.                   |4                     |
|2010-06-30|T       |Af.Ac.Trabalho                   |1                     |
|2010-08-31|T       |Af.Ac.Trab